<a href="https://colab.research.google.com/github/LasyaMoravineni/AI_chatbot/blob/main/AI_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Gradio
!pip install gradio

# Import required libraries
import json
import pandas as pd
import nltk
import string
import gradio as gr
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')


In [ ]:
# Mental health intents dataset (you can customize further)
intents = {
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hi", "Hello", "Hey", "Is anyone there?", "Good morning", "Good evening"],
      "responses": ["Hello, I'm here for you. How can I help today?"]
    },
    {
      "tag": "sadness",
      "patterns": ["I'm feeling sad", "I am depressed", "Life is hard", "I feel low", "I'm down", "Crying a lot lately"],
      "responses": ["I'm really sorry you're feeling this way. Do you want to talk more about what's been going on?"]
    },
    {
      "tag": "anxiety",
      "patterns": ["I'm anxious", "I can't calm down", "Feeling nervous", "I'm panicking", "My heart is racing"],
      "responses": ["Try taking a few deep breaths. Would you like me to guide you through a calming exercise?"]
    },
    {
      "tag": "loneliness",
      "patterns": ["I feel alone", "Nobody understands me", "I'm lonely", "No one to talk to"],
      "responses": ["You’re not alone right now — I'm here with you. Want to share what's on your mind?"]
    },
    {
      "tag": "burnout",
      "patterns": ["I'm exhausted", "I'm burnt out", "Too much pressure", "I can't handle this workload"],
      "responses": ["It’s okay to take a break. Rest is essential, not a weakness. Want to talk through it?"]
    },
    {
      "tag": "self_esteem",
      "patterns": ["I hate myself", "I'm not good enough", "I always fail", "No one likes me"],
      "responses": ["You matter. Everyone struggles sometimes, but that doesn't define your worth. Want to talk more?"]
    },
    {
      "tag": "suicidal_thoughts",
      "patterns": ["I want to die", "I don't want to live", "Thinking of ending it", "No reason to go on"],
      "responses": [
        "I'm so sorry you're feeling this way. You're not alone, and your life is valuable. Please consider speaking to a mental health professional or calling a helpline."
      ]
    },
    {
      "tag": "motivation",
      "patterns": ["I need motivation", "Feeling unmotivated", "Can't keep going", "Nothing's working out"],
      "responses": [
        "You're stronger than you think. Every step forward, even the smallest one, is progress. I'm cheering for you."
      ]
    },
    {
      "tag": "sleep_issues",
      "patterns": ["I can't sleep", "I'm having trouble sleeping", "Restless at night", "Keep waking up"],
      "responses": ["Poor sleep can be tough. Would you like some tips on better sleep habits?"]
    },
    {
      "tag": "coping_strategies",
      "patterns": ["How do I deal with stress?", "Any coping strategies?", "How to manage anxiety?", "Help me handle pressure"],
      "responses": ["There are many healthy coping strategies like journaling, deep breathing, or talking to someone. Want to try one together?"]
    },
    {
      "tag": "thanks",
      "patterns": ["Thanks", "Thank you", "Much appreciated", "You're helpful"],
      "responses": ["You're welcome. I'm glad I could be here for you."]
    },
    {
      "tag": "goodbye",
      "patterns": ["Bye", "Goodbye", "See you", "Talk later", "I'm done for now"],
      "responses": ["Take care. I’m always here if you need someone to talk to again."]
    }
  ]
}
# Save intents to JSON file
with open("intents.json", "w") as f:
    json.dump(intents, f)


In [ ]:
# Load intents
with open("intents.json") as file:
    data = json.load(file)

texts = []
labels = []
responses_dict = {}

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        texts.append(pattern)
        labels.append(intent["tag"])
    responses_dict[intent["tag"]] = intent["responses"]

df = pd.DataFrame({"Text": texts, "Intent": labels})

# Preprocessing
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in string.punctuation]
    return " ".join(tokens)

df['Cleaned'] = df['Text'].apply(preprocess)


In [ ]:
# Vectorize and train
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Cleaned'])

encoder = LabelEncoder()
y = encoder.fit_transform(df['Intent'])

model = LogisticRegression()
model.fit(X, y)


In [ ]:
def chatbot_response(user_input):
    cleaned = preprocess(user_input)
    vector = vectorizer.transform([cleaned])
    prediction = model.predict(vector)
    intent = encoder.inverse_transform(prediction)[0]
    response = responses_dict[intent]
    return response[0]


In [ ]:
import gradio as gr

def chatbot_response(user_input):
    cleaned = preprocess(user_input)
    vector = vectorizer.transform([cleaned])
    prediction = model.predict(vector)
    intent = encoder.inverse_transform(prediction)[0]
    response = responses_dict[intent]
    return response[0]

with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Mental Health Chatbot\n_I'm here to listen and support you._")

    # Inject CSS for button styling
    gr.HTML("""
    <style>
        .send-button button {
            background-color: #4CAF50 !important; /* Green */
            color: white !important;
            padding: 2px 12px !important;
            font-size: 20px !important;
            height: 40px !important;
            border: none;
            border-radius: 6px;
            cursor: pointer;
        }
        .send-button button:hover {
            background-color: #45a049 !important; /* Slightly darker green on hover */
        }
    </style>
    """)

    chatbot = gr.Chatbot(label="Chat", height=400, type="messages")

    with gr.Row(equal_height=True):
        user_input = gr.Textbox(
            placeholder="Type your message...",
            show_label=False,
            scale=10
        )
        send_button = gr.Button(
            value="Send ➤",
            elem_classes=["send-button"],  # Class to target in CSS
            scale=1
        )

    def respond(message, chat_history):
        bot_reply = chatbot_response(message)
        chat_history.append({"role": "user", "content": message})
        chat_history.append({"role": "assistant", "content": bot_reply})
        return "", chat_history

    user_input.submit(respond, [user_input, chatbot], [user_input, chatbot])
    send_button.click(respond, [user_input, chatbot], [user_input, chatbot])

demo.launch()

